# **Quality check segmentations**

***Prior to this notebook, you should have already completed the organelle and masks segmentations using notebooks 1.0 through 1.7 and/or batch_processing_segmentations OR completed segmentations using the `infer-subc` plugin for Napari.***

### **Purpose**
The purpose of this notebook is to:
1. Visually inspect the segmentation outputs for each image in your dataset (***optional***)

    We highly recommend going through this process. It is especially important for datasets where errors in the segmentations can have large impacts on the downstream statistical outcomes (e.g., datasets that do NOT have ~100+ images per condition for example).
2. Separation of the `masks` file into separate `cell` and `nuc` segmentation files (***required***)

    This is required for downstream analysis in `infer-subc` Part 2. For some background, this step is required due to technical limitations of the Napari plugin; each workflow can only output one file, but the segmentations of the nucleus and cell are derived from the same workflows.

### ➡️ **Input**
The following files will be used as input in this notebook:
1. "Raw" confocal microscopy image (".tiff", ".tif", or ".czi") where each channel of the image represents one of the organelles being segmented
2. The segmentation files created in `infer-subc` Notebooks or Napari plugin or derived from alternative segmentation tools.

You will also need the following:
1. An empty folder to save segmentation files after editing
2. An empty folder to save the FINAL segmentation dataset (e.g., combining edited files with original segmentation files that did not need editing)

### **Output** ➡️
The output from this notebook will include:
1. A new folder containing any edited mask files saved as ".tiff"
2. A new folder containing ALL FINAL segmentation files that are ready for quantification

-----

### 👣 **Summary of steps**  

➡️ **QUALITY CHECK SEGMENTATIONS**
- **`Step 1`** - Define the paths to your data and final output location

- **`Step 2`** - Import and visualize organelle and region segmentations

- **`Step 3`** - Edit individual segmentations (***optional***) and quality check segmentations before quantification

- **`Step 4`** - Save organelle and region segmentations into specified folder

    This step will also save the cell and nuclei segmentations as separate files (***required***)

---------------------
## **IMPORTS**

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

import napari
viewer = napari.Viewer()

from infer_subc.core.file_io import (read_czi_image,
                                        export_inferred_organelle,
                                        import_inferred_organelle,
                                        list_image_files,)
from infer_subc.core.img import apply_mask, label_bool_as_uint16, label
from infer_subc.utils.batch import QC_filter

%load_ext autoreload
%autoreload 2

-----

## **QUALITY CHECK OF SEGMENTATIONS**

### **`STEP 1` - Define the paths to your data**

#### &#x1F6D1; &#x270D; **User Input Required:**

Please specify the following information about your data: 
- `raw_path`: A string of the path to your raw (e.g., intensity) images that were used as the input for segmentation
- `seg_path`: A string of the path where the segmentation outputs will be saved
- `location_tosave_edited_segmentations`: A string to a empty folder where you will save segmentation files after editing them.
- `location_tosave_fullset_gooddata`: A string to a empty folder that will contain all of the FINAL segmentation files (e.g., it will combine the newly saved edited segmentation files with any of the original segmentation files that did not need editing)
- `raw_file_type`: The raw file type (e.g., ".tiff" or ".czi")
- `seg_file_type`: The segmentation file type (e.g., ".tiff" or ".tif" if you were using infer-subc notebooks or Napari plugin)
- `{seg-name}_suffix`: the ending of your segmentation file name; this usually just includes the segmentation suffix (e.g., 'lyso'), but may also include additional text if the settings file in Napari or the 'name_suffix' in the batch_process_segmentation notebook contained additional text. *If the segmentation listed was not included in your analysis, type `None`*.
- `{seg-name}_ch`: the index of the channel in the raw intensity image used to generate the associated segmentation. Channel indexes begin at 0 for channel 1.

<mark>build in sample data use here for testing out this notebook with the sample data<mark>

In [ ]:
#### USER INPUT REQUIRED ###
raw_data = "E:/Experiments (C2-117 - current)/C2-121/C2-121_deconvolution"
seg_path = "E:/Experiments (C2-117 - current)/C2-121/20230921_C2-121_3D-analysis/20230921_C2-121_segmentation"

location_tosave_edited_segmentations = "E:/Experiments (C2-117 - current)/C2-121/20230921_C2-121_3D-analysis/20240102_C2-121_segmentation-edits"
location_tosave_fullset_gooddata = "E:/Experiments (C2-117 - current)/C2-121/20230921_C2-121_3D-analysis/C2-121_good-segs"

raw_file_type = ".tiff"
seg_file_type = ".tiff"

lyso_ch = 3
mito_ch = 4
golgi_ch = 2
perox_ch = 5
ER_ch = 1
LD_ch = 0

mask_suffix = "masks_A"
lyso_suffix = "lyso"
mito_suffix = "mito"
golgi_suffix = "golgi"
perox_suffix = "perox"
ER_suffix = "ER"
LD_suffix = "LD"
# eventually transition to this formatting: 
# suffix = ['masks_A', 'lyso', 'mito', 'golgi', 'perox', 'ER', 'LD']
# ch = [None, 1, 2, 3, 4, 5, 0]

#### &#x1F6D1; &#x270D; ***Optional* User Input Required:**

&#x1F453; **FYI:** If you ran the batch processing more than once, you can include information about the additional segmentation location and file suffixes here. If you do **NOT** have a second file location, for each object below, please indicate: `None`

***Important:** Only one location for each segmentation type will be used. If `lyso_suffix` and `lyso_suffix_2` are both specified, **ONLY** `lyso_suffix_2` will be visualized below.*

In [ ]:
#### Optional - USER INPUT REQUIRED ###
seg_path_2 = None

mask_suffix_2 = None
lyso_suffix_2 = None
mito_suffix_2 = None
golgi_suffix_2 = None
perox_suffix_2 = None
ER_suffix_2 = None
LD_suffix_2 = None

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block will print the list of raw file names in your raw_path. You will want to review each file in the list for accurate segmentations. The next set of code blocks will walk you how to do this.

In [ ]:
raw_file_list = list_image_files(Path(raw_data),".tiff")

pd.set_option('display.max_colwidth', None)
pd.DataFrame({"Image Name":raw_file_list})

### **`STEP 2` - Import and visualize organelle and region segmentations**

#### &#x1F6D1; &#x270D; **User Input Required:**
&#x1F53C; Use the list  above to determine the index of the image you would like to look at. Eventually, you should look at all images. The pipeline below will indicate when you need to return here to repeat for the next image in the list.

In [ ]:
#### USER INPUT REQUIRED ###
num = 11

#### &#x1F3C3; **Run code; no user input required**
&#x1F453; **FYI:** This code block will import the segmentation files associated to the raw image you selected above. The raw image and segmentations will be included in a Napari viewer for visual inspection.

In [ ]:
raw_img_data, raw_meta_dict = read_czi_image(raw_file_list[num])
print("Image name:")
print(raw_meta_dict['name'][0].split(" :: ")[0])

if mask_suffix is not None:
    mask_seg = import_inferred_organelle(mask_suffix, raw_meta_dict, Path(seg_path), seg_file_type)
    cell_seg = mask_seg[1]
    nuc_seg = mask_seg[0]
if lyso_suffix is not None:
    lyso_seg = import_inferred_organelle(lyso_suffix, raw_meta_dict, Path(seg_path), seg_file_type)
if mito_suffix is not None:
    mito_seg = import_inferred_organelle(mito_suffix, raw_meta_dict, Path(seg_path), seg_file_type)
if golgi_suffix is not None:
    golgi_seg = import_inferred_organelle(golgi_suffix, raw_meta_dict, Path(seg_path), seg_file_type)
if perox_suffix is not None:
    perox_seg = import_inferred_organelle(perox_suffix, raw_meta_dict, Path(seg_path), seg_file_type)
if ER_suffix is not None:
    ER_seg = import_inferred_organelle(ER_suffix, raw_meta_dict, Path(seg_path), seg_file_type)
if LD_suffix is not None:
    LD_seg = import_inferred_organelle(LD_suffix, raw_meta_dict, Path(seg_path), seg_file_type)

if seg_path_2 is not None:
    if mask_suffix_2 is not None:
        mask_seg = import_inferred_organelle(mask_suffix_2, raw_meta_dict, Path(seg_path_2), seg_file_type)
        cell_seg = mask_seg[1]
        nuc_seg = mask_seg[0]
        mask_suffix = mask_suffix_2
    if lyso_suffix_2 is not None:
        lyso_seg = import_inferred_organelle(lyso_suffix_2, raw_meta_dict, Path(seg_path_2), seg_file_type)
        lyso_suffix = lyso_suffix_2
    if mito_suffix_2 is not None:
        mito_seg = import_inferred_organelle(mito_suffix_2, raw_meta_dict, Path(seg_path_2), seg_file_type)
        mito_suffix = mito_suffix_2
    if golgi_suffix_2 is not None:
        golgi_seg = import_inferred_organelle(golgi_suffix_2, raw_meta_dict, Path(seg_path_2), seg_file_type)
        golgi_suffix = golgi_suffix_2
    if perox_suffix_2 is not None:
        perox_seg = import_inferred_organelle(perox_suffix_2, raw_meta_dict, Path(seg_path_2), seg_file_type)
        perox_suffix = perox_suffix_2
    if ER_suffix_2 is not None:
        ER_seg = import_inferred_organelle(ER_suffix_2, raw_meta_dict, Path(seg_path_2), seg_file_type)
        ER_suffix = ER_suffix_2
    if LD_suffix_2 is not None:
        LD_seg = import_inferred_organelle(LD_suffix_2, raw_meta_dict, Path(seg_path_2), seg_file_type)
        LD_suffix = LD_suffix_2

viewer.layers.clear()
if np.any(LD_seg):
    viewer.add_image(raw_img_data[LD_ch], name=f'{LD_suffix}_raw', blending='additive')
    viewer.add_image(LD_seg, opacity=0.3, name=f'{LD_suffix}_seg', colormap='magenta')
if np.any(ER_seg):
    viewer.add_image(raw_img_data[ER_ch], name=f'{ER_suffix}_raw', blending='additive')
    viewer.add_image(ER_seg, opacity=0.3, name=f'{ER_suffix}_seg', colormap='red')
if np.any(golgi_seg):
    viewer.add_image(raw_img_data[golgi_ch], name=f'{golgi_suffix}_raw', blending='additive')
    viewer.add_image(golgi_seg, opacity=0.3, name=f'{golgi_suffix}_seg', colormap='yellow')
if np.any(lyso_seg):
    viewer.add_image(raw_img_data[lyso_ch], name=f'{lyso_suffix}_raw', blending='additive')
    viewer.add_image(lyso_seg, opacity=0.3, name=f'{lyso_suffix}_seg', colormap='cyan')
if np.any(mito_seg):
    viewer.add_image(raw_img_data[mito_ch], name=f'{mito_suffix}_raw', blending='additive')
    viewer.add_image(mito_seg, opacity=0.3, name=f'{mito_suffix}_seg', colormap='green')
if np.any(perox_seg):
    viewer.add_image(raw_img_data[perox_ch], name=f'{perox_suffix}_raw', blending='additive')
    viewer.add_image(perox_seg, opacity=0.3, name=f'{perox_suffix}_seg', colormap='bop orange')
if np.any(mask_seg):
    viewer.add_image(mask_seg, opacity=0.3, name=f'{mask_suffix}_seg')
    viewer.add_image(cell_seg, opacity=0.3, name="cell_seg")
    viewer.add_image(nuc_seg, opacity=0.3, name="nuc_seg")

> #### &#x1F6D1; **STOP: Use the `Napari` window to review all of the segmentations for this image.** &#x1F50E;
>
> **At this point, take note of which segmentations need to be edited, if any. Once you are finished reviewing the segmentations for this image, continue on to the next sections to:**
> 1) Edit the segmentation (if necessary), then
> 2) Save the final set of segmentations for this image in a new folder. This step will also save the `cell` and `nuc` segmentation files separately in the final analysis location.

### **`STEP 3` - Edit *(optional)* and quality check segmentations**

#### &#x1F6D1; &#x270D; **User Input Required:**
Now that you visually inspected your image and the segmentation output files, indicate below which of the segmentations you would like to edit, if any, by typing one of the following options for each segmentation type:
- `True`: for any files you want to edit
- `False`: for any that you do not want to edit OR if you did not include that segmentation in your analysis

Additionally, you have the option to choose a filter to apply to your cells that may not meet assumptions specified below. Indicate the filter, if any, you would like the computer to apply to correct for the assumptions by typing one of the following options for each segmentation type:
- `"largest"`: only the largest nonzero object will be considered
- `"brightest"`: only the brightest object from a composite of all layers of the raw image will be considered
- `any integer value`: only objects that are larger than the specified number of voxels will be considered
- `None`: no filtering will be applied

In [ ]:
#### USER INPUT REQUIRED ###
edit_cell = False
filter_cell = None
edit_nuc = False
filter_nuc = None
edit_LD = False 
edit_ER = False
edit_golgi = False
edit_lyso = False
edit_mito = False
edit_perox = False

#### &#x1F3C3; **Run code; no user input required** 
&#x1F440; ***See code block output for instructions:***

For the following blocks of code, run them without any changes, and do not edit the naming of the layers in Napari. Read the output of each cell for further instructions!!

**Important:** These blocks of code check your segmentations for the following assumptions:
- One cell in the cell mask file
- One nuclei within the specified cell mask
- The ER is a single object (if it is not, the code will force it to be labeled as one object automatically; no additional user editting will be required)

If these assumptions are not met in your image, the computer will attempt to automatically correct your image based on the filter_object value above (where object refers to the individual object). If you do not wish the computer to make any automatic corrections, set the filter_object value to None. Additional instructions will be included in the output of the code block.

In [ ]:
if edit_cell is True:
    viewer.layers.clear()
    viewer.add_image(raw_img_data)
    viewer.add_labels(cell_seg)
    print("Head to the Napari window!")
    print("You can edit your segmentation as needed there, then run the next block of code.")
elif edit_cell is False:
    print("Continue - run the next block of code")
else:
    print("There is an error somewhere!")

In [ ]:
# QC for object number per image
if (np.size(np.unique(label(viewer.layers['cell_seg'].data))) > 2):
    print("Your cell mask segmentation contains MORE THAN ONE cell mask object. For quantification, you must only have ONE cell mask object, attempting to correct this automatically...")
    
    filtered_cell_seg = QC_filter(viewer.layers['cell_seg'].data, raw_img_data, method=filter_cell)

    if np.size(np.unique(filtered_cell_seg)) == 2:
        print("The image has been processed to automatical remove any small objects.")
        cell_seg = filtered_cell_seg
        viewer.add_labels(filtered_cell_seg, name="filtered_cell_seg")
        print("Head to the Napari window to see your filtered cell segmentation output!")
        print("If satisfied, run the next block of code.")
        print("Note: if further edits are desired, please edit the cell_seg layer instead of the filtered_cell_seg layer, then rerun this block of code.")
        
    elif np.size(np.unique(filtered_cell_seg)) > 2:
        print("We tried to remove small objects, but there are still multiple cell mask objects in the image. Please try other 'filter_cell' values above or edit the segmentation manually in Napari.")
        viewer.add_labels(filtered_cell_seg, name="filtered_cell_seg")
        print("Head to the Napari window to see your filtered cell segmentation output!")
        print("Note: any edits made to the filtered_cell_seg layer will not be saved. Direct desired edits to the cell_seg layer instead, then rerun this block of code.")

    else:
        print("There are no cell mask objects in the image. Please check your segmentation files, and/or filter_cell value and try again. If necessary, rerun the previous code block to reset the image.")

In [ ]:
if edit_nuc is True:
    viewer.layers.clear()
    viewer.add_image(raw_img_data)
    viewer.add_labels(nuc_seg)
    print("Head to the Napari window!")
    print("You can edit your segmentation as needed there, then run the next block of code.")
elif edit_nuc is False:
    print("Continue - run the next block of code")
else:
    print("There is an error somewhere!")

In [ ]:
# QC for object number per image
if (np.size(np.unique(label(apply_mask(viewer.layers['nuc_seg'].data, cell_seg)))) > 2):
    print("Your nucleus segmentation contains MORE THAN ONE nucleus object within the cell mask. For quantification, you must only have ONE nucleus mask object, attempting to correct this automatically...")
    
    filtered_nuc_seg = QC_filter(viewer.layers['nuc_seg'].data, raw_img_data, method=filter_nuc)

    if np.size(np.unique(filtered_nuc_seg)) == 2:
        print("The image has been processed to automatical remove any small objects.")
        nuc_seg = filtered_nuc_seg
        viewer.add_labels(filtered_nuc_seg, name="filtered_nuc_seg")
        print("Head to the Napari window to see your filtered nuc segmentation output!")
        print("If satisfied, run the next block of code.")
        print("Note: if further edits are desired, please edit the nuc_seg layer instead of the filtered_nuc_seg layer, then rerun this block of code.")
        
    elif np.size(np.unique(filtered_nuc_seg)) > 2:
        print("We tried to remove small objects, but there are still multiple nucleus objects in the image. Please try other 'filter_nuc' values above or edit the segmentation manually in Napari.")
        viewer.add_labels(filtered_nuc_seg, name="filtered_nuc_seg")
        print("Head to the Napari window to see your filtered nucleus segmentation output!")
        print("Note: any edits made to the filtered_nuc_seg layer will not be saved. Direct desired edits to the nuc_seg layer instead, then rerun this block of code.")
    else:
        print("There are no nucleus objects in the image. Please check your segmentation files, and/or filter_nuc value and try again. If necessary, rerun the previous code block to reset the image.")

else:
    print("Continue - run the next block of code")

In [ ]:
if edit_LD is True:
    viewer.layers.clear()
    viewer.add_image(raw_img_data)
    viewer.add_labels(LD_seg)
    print("Head to the Napari window!")
    print("You can edit your segmentation as needed there, then run the next block of code.")
elif edit_LD is False:
    print("Continue - run the next block of code")
else:
    print("There is an error somewhere!")

In [ ]:
if edit_LD is True:
    LD_seg = viewer.layers['LD_seg'].data
print("Your LD segmentation been has stored to local memory.")
print("Continue - run the next block of code")

In [ ]:
if np.size(np.unique(ER_seg)) > 2:
    print("Your ER segmentation contains MORE THAN ONE ER object. For infer-subc quantification, we assume that the ER is one continuous object.")
    ER_seg = label_bool_as_uint16(ER_seg)
    print("Your ER segmentation has been updated. All of the objects have been given the same ID number and will be considered one object during quantification.")
elif edit_ER is True:
    viewer.add_image(raw_img_data)
    viewer.add_labels(ER_seg)
    print("Head to the Napari window!")
    print("You can edit your segmentation as needed there.")
elif edit_ER is False:
    print("Continue - run the next block of code")
else:
    print("There is an error somewhere!")

In [ ]:
if edit_ER is True:
    ER_seg = viewer.layers['ER_seg'].data
print("Your ER segmentation been has stored to local memory.")
print("Continue - run the next block of code")

In [ ]:
if edit_golgi is True:
    viewer.layers.clear()
    viewer.add_image(raw_img_data)
    viewer.add_labels(golgi_seg)
    print("Head to the Napari window!")
    print("You can edit your segmentation as needed there.")
elif edit_golgi is False:
    print("Continue - run the next block of code")
else:
    print("There is an error somewhere!")

In [ ]:
if edit_golgi is True:
    golgi_seg = viewer.layers['golgi_seg'].data
print("Your golgi segmentation been has stored to local memory.")
print("Continue - run the next block of code")

In [ ]:
if edit_lyso is True:
    viewer.layers.clear()
    viewer.add_image(raw_img_data)
    viewer.add_labels(lyso_seg)
    print("Head to the Napari window!")
    print("You can edit your segmentation as needed there.")
elif edit_lyso is False:
    print("Continue - run the next block of code")
else:
    print("There is an error somewhere!")

In [ ]:
if edit_lyso is True:
    lyso_seg = viewer.layers['lyso_seg'].data
print("Your lyso segmentation been has stored to local memory.")
print("Continue - run the next block of code")

In [ ]:
if edit_mito is True:
    viewer.layers.clear()
    viewer.add_image(raw_img_data)
    viewer.add_labels(mito_seg)
    print("Head to the Napari window!")
    print("You can edit your segmentation as needed there.")
elif edit_mito is False:
    print("Continue - run the next block of code")
else:
    print("There is an error somewhere!")

In [ ]:
if edit_mito is True:
    mito_seg = viewer.layers['mito_seg'].data
print("Your mito segmentation been has stored to local memory.")
print("Continue - run the next block of code")

In [ ]:
if edit_perox is True:
    viewer.layers.clear()
    viewer.add_image(raw_img_data)
    viewer.add_labels(perox_seg)
    print("Head to the Napari window!")
    print("You can edit your segmentation as needed there.")
elif edit_perox is False:
    print("Continue - run the next block of code")
else:
    print("There is an error somewhere!")

In [ ]:
if edit_perox is True:
    perox_seg = viewer.layers['perox_seg'].data
print("Your perox segmentation been has stored to local memory.")
print("Continue - run the next block of code")

### **`STEP 4` - Save final organelle and region segmentations into specified folder**

#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block combines the newly editted segmentation file(s) with any of the segmentation files that did not need editting. They are saved into a NEW file location (specified in the beginning of this notebook) that will be used as the input for `Part 2 - Organelle Quantification`.

In [ ]:
if edit_cell is True:
    out_file_n = export_inferred_organelle(cell_seg, "cell", raw_meta_dict, Path(location_tosave_edited_segmentations))
    out_file_n = export_inferred_organelle(cell_seg, "cell", raw_meta_dict, Path(location_tosave_fullset_gooddata))
elif edit_cell is False:
    cell_seg = mask_seg[1]
    out_file_n = export_inferred_organelle(cell_seg, "cell", raw_meta_dict, Path(location_tosave_fullset_gooddata))
else:
    print("There is an error somewhere!")

In [ ]:
if edit_nuc is True:
    out_file_n = export_inferred_organelle(nuc_seg, "nuc", raw_meta_dict, Path(location_tosave_edited_segmentations))
    out_file_n = export_inferred_organelle(nuc_seg, "nuc", raw_meta_dict, Path(location_tosave_fullset_gooddata))
elif edit_cell is False:
    nuc_seg = mask_seg[0]
    out_file_n = export_inferred_organelle(nuc_seg, "nuc", raw_meta_dict, Path(location_tosave_fullset_gooddata))
else:
    print("There is an error somewhere!")

In [ ]:
if edit_LD is True:
    out_file_n = export_inferred_organelle(LD_seg, "LD", raw_meta_dict, Path(location_tosave_edited_segmentations))
    out_file_n = export_inferred_organelle(LD_seg, "LD", raw_meta_dict, Path(location_tosave_fullset_gooddata))
elif edit_LD is False:
    out_file_n = export_inferred_organelle(LD_seg, "LD", raw_meta_dict, Path(location_tosave_fullset_gooddata))
else:
    print("There is an error somewhere!")

In [ ]:
if edit_ER is True:
    out_file_n = export_inferred_organelle(ER_seg, "ER", raw_meta_dict, Path(location_tosave_edited_segmentations))
    out_file_n = export_inferred_organelle(ER_seg, "ER", raw_meta_dict, Path(location_tosave_fullset_gooddata))
elif edit_ER is False:
    out_file_n = export_inferred_organelle(ER_seg, "ER", raw_meta_dict, Path(location_tosave_fullset_gooddata))
else:
    print("There is an error somewhere!")

In [ ]:
if edit_golgi is True:
    out_file_n = export_inferred_organelle(golgi_seg, "golgi", raw_meta_dict, Path(location_tosave_edited_segmentations))
    out_file_n = export_inferred_organelle(golgi_seg, "golgi", raw_meta_dict, Path(location_tosave_fullset_gooddata))
elif edit_golgi is False:
    out_file_n = export_inferred_organelle(golgi_seg, "golgi", raw_meta_dict, Path(location_tosave_fullset_gooddata))
else:
    print("There is an error somewhere!")

In [ ]:
if edit_lyso is True:
    out_file_n = export_inferred_organelle(lyso_seg, "lyso", raw_meta_dict, Path(location_tosave_edited_segmentations))
    out_file_n = export_inferred_organelle(lyso_seg, "lyso", raw_meta_dict, Path(location_tosave_fullset_gooddata))
elif edit_lyso is False:
    out_file_n = export_inferred_organelle(lyso_seg, "lyso", raw_meta_dict, Path(location_tosave_fullset_gooddata))
else:
    print("There is an error somewhere!")

In [ ]:
if edit_mito is True:
    out_file_n = export_inferred_organelle(mito_seg, "mito", raw_meta_dict, Path(location_tosave_edited_segmentations))
    out_file_n = export_inferred_organelle(mito_seg, "mito", raw_meta_dict, Path(location_tosave_fullset_gooddata))
elif edit_mito is False:
    out_file_n = export_inferred_organelle(mito_seg, "mito", raw_meta_dict, Path(location_tosave_fullset_gooddata))
else:
    print("There is an error somewhere!")

In [ ]:
if edit_perox is True:
    out_file_n = export_inferred_organelle(perox_seg, "perox", raw_meta_dict, Path(location_tosave_edited_segmentations))
    out_file_n = export_inferred_organelle(perox_seg, "perox", raw_meta_dict, Path(location_tosave_fullset_gooddata))
elif edit_perox is False:
    out_file_n = export_inferred_organelle(perox_seg, "perox", raw_meta_dict, Path(location_tosave_fullset_gooddata))
else:
    print("There is an error somewhere!")

> ### 🔁🔁 **Repeat `Steps 2-4` above for *EACH IMAGE* in your dataset.**  
>
> You've now completed the quality check for one image. Repeat these steps for each image in your dataset by selecting the next image in the list at the beginning of `Step 2` above.

-----
### 🎉 **CONGRATULATIONS!! You've successfully completed `infer-subc Part 1 - Segmentation Workflows`.**

Continue on to `infer-subc Part 2 - Organelle Quantification`. An overview is included in the [`2.0_quantification_setup`](..\part_2_quantification\2.0_quantification_setup.ipynb) notebook.